In [470]:
import pandas as pd

# Load CSVs downloaded from kaggle
team_details = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/team_details.csv')
pitstops = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/pitstops.csv')
race_details = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/race_details.csv')
driver_standings = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/driver_standings.csv')
driver_details = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/driver_details.csv')
starting_grids = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/starting_grids.csv')
race_summaries = pd.read_csv('/kaggle/input/formula-1-official-data-19502022/race_summaries.csv')


# Merge driver details with race details
df = pd.merge(race_details, driver_details, how='inner', on=['Driver', 'Grand Prix', 'Year', 'Car'], suffixes=('_race', '_details'))

# Merge with driver standings
df = pd.merge(df, driver_standings, how='inner', on=['Driver', 'Year', 'DriverCode', 'Car'], suffixes=('', '_standings'))

# Merge with starting grids
df = pd.merge(df, starting_grids, how='inner', on=['Driver', 'Grand Prix', 'Year', 'DriverCode', 'Car'], suffixes=('', '_grid'))

# Merge with race summaries
df = pd.merge(df, race_summaries, how='inner', on=['Grand Prix', 'Year', 'Car', 'Date'], suffixes=('', '_summary'))

# Merge with team details
df = pd.merge(df, team_details, how='inner', on=['Grand Prix', 'Year', 'Date'], suffixes=('', '_team'))

# Merge with pitstops
df = pd.merge(df, pitstops, how='inner', on=['Driver', 'Grand Prix', 'Year', 'DriverCode', 'Car'], suffixes=('', '_pitstop'))

# saving merged race data to further clean it
df.to_csv('merged_race_data.csv', index=False)

In [471]:
df.shape

(20359, 34)

# DATA CLEANING

Selecting races only after 2010

In [473]:
# selecting only races after 2010
df = df[df["Year"]>2010]

Dropping duplicates

In [474]:
# removing duplicate values
df.drop_duplicates(inplace=True)
print(df.shape)
df.isna().sum()

(9814, 34)


Pos                 0
No                  0
Driver              0
Car                 0
Laps                0
Time/Retired        0
PTS_race            0
Year                0
Grand Prix          0
Detail              0
DriverCode          0
Date                0
PTS_details         0
Race Position       0
Pos_standings       0
Nationality         0
PTS                 0
Detail_grid         0
No_grid             0
Pos_grid            0
Time              464
Winner              0
Laps_summary        0
Time_summary        0
WinnerCode          0
PTS_team            0
Team                0
Stops               0
No_pitstop          0
Lap                 0
Time of day         0
Time_pitstop        0
Total               0
Detail_pitstop      0
dtype: int64

## Cleaning each feature individually 

- imputing null values, changing their types, etc.

`Pos` has "1" to "19" values except the "NC" value which we replace with "0" since it is "Not Classified"

In [475]:
df.loc[df['Pos'] == 'NC', 'Pos'] = '0'
df['Pos'] = df['Pos'].astype(int)

In [476]:
df['Detail'].unique()

array(['Race-Result'], dtype=object)

Since `Detail` has only one value it makes sense to just drop it altogether

In [477]:
df.drop(['Detail'], axis=1, inplace=True)

### Changing date type columns from str to datetime

In [478]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df["Time_summary"] = pd.to_timedelta(df["Time_summary"])
df["Time of day"] = pd.to_timedelta(df["Time of day"])

Changing the DNF race position to be max+1 (which is 20)

In [479]:
df.loc[df['Race Position'] == 'DNF', 'Race Position'] = '21'
df['Race Position'] = df['Race Position'].astype(int)
df['Race Position'].unique()

array([ 1,  5,  4,  2,  3, 21, 15, 16, 12,  6, 11,  9, 10,  7, 14,  8, 13])

Pos_standings is a str of numbers that needs to be changed to int

In [480]:
# changin type to int from str
df['Pos_standings'] = df['Pos_standings'].astype(int)

In [481]:
print(df["Detail_grid"].unique())
# Dropping Detail_grid since it has only one value
df.drop(['Detail_grid'], axis=1, inplace=True)

['Starting-Grid']


Plotting other unique values to clean

In [482]:
df["No_grid"].unique()

array([ 1,  2,  3,  4,  5,  6,  8,  7, 18, 10,  9, 44, 77, 33, 16, 23, 26,
       11, 31, 14, 55, 63])

In [483]:
df["Pos_grid"].unique()

array([ 1,  3,  2,  7,  4,  8, 12,  9, 10, 11, 13,  5,  6, 20, 22, 21, 14,
       16])

`Time` column has some null values for drivers "NOR" and "RUS".
But since the column is irrelevant to us, we drop it

In [485]:
# dropping time because it is starting time (which is irrelevant)
df.drop(['Time'], axis=1, inplace=True)

In [486]:
df["Team"].unique()

array(['Ferrari', 'Force India Mercedes', 'Lotus Renault',
       'Mclaren Mercedes', 'Mercedes', 'Red Bull Racing Renault',
       'Renault', 'Sauber Ferrari', 'Str Ferrari', 'Virgin Cosworth',
       'Williams Cosworth', 'Hrt Cosworth', 'Caterham Renault',
       'Marussia Cosworth', 'Williams Renault', 'Marussia Ferrari',
       'Str Renault', 'Williams Mercedes', 'Lotus Mercedes',
       'Mclaren Honda', 'Haas Ferrari', 'Mrt Mercedes',
       'Red Bull Racing Tag Heuer', 'Toro Rosso Ferrari', 'Toro Rosso',
       'Force India Sahara', 'Mclaren Renault',
       'Scuderia Toro Rosso Honda', 'Alfa Romeo Racing Ferrari',
       'Racing Point Bwt Mercedes', 'Red Bull Racing Honda',
       'Alphatauri Honda', 'Alpine Renault', 'Aston Martin Mercedes',
       'Alfa Romeo Ferrari', 'Alphatauri Rbpt',
       'Aston Martin Aramco Mercedes', 'Red Bull Racing Rbpt'],
      dtype=object)

A lot of teams are repeated here.
Creating a mapping to map duplicate teams to one team to increase our accuracy

In [487]:
# Create a mapping dictionary
mapping_dict = {
    # Mapping each group to its first mentioned common name
    'Force India Sahara': 'Force India Mercedes',
    'Racing Point Bwt Mercedes': 'Force India Mercedes',

    'Red Bull Racing Renault': 'Red Bull Racing',
    'Red Bull Racing Tag Heuer': 'Red Bull Racing',
    'Red Bull Racing Honda': 'Red Bull Racing',
    'Red Bull Racing Rbpt': 'Red Bull Racing',

    'Str Ferrari': 'Scuderia Toro Rosso',
    'Toro Rosso Ferrari': 'Scuderia Toro Rosso',
    'Toro Rosso': 'Scuderia Toro Rosso',
    'Scuderia Toro Rosso Honda': 'Scuderia Toro Rosso',
    'Alphatauri Honda': 'Scuderia Toro Rosso',
    'Alphatauri Rbpt': 'Scuderia Toro Rosso',

    'Virgin Cosworth': 'Marussia',
    'Marussia Cosworth': 'Marussia',
    'Marussia Ferrari': 'Marussia',
    'Mrt Mercedes': 'Marussia',

    'Lotus Renault': 'Lotus',
    'Lotus Mercedes': 'Lotus',

    'Alfa Romeo Racing Ferrari': 'Alfa Romeo',
    'Alfa Romeo Ferrari': 'Alfa Romeo',

    'Aston Martin Mercedes': 'Aston Martin',
    'Aston Martin Aramco Mercedes': 'Aston Martin',

    'Alpine Renault': 'Renault',
    'Renault': 'Renault'
}

# Replace values in the column based on the mapping dictionary
df['Team'] = df['Team'].replace(mapping_dict)

Result

In [488]:
df["Team"].unique()

array(['Ferrari', 'Force India Mercedes', 'Lotus', 'Mclaren Mercedes',
       'Mercedes', 'Red Bull Racing', 'Renault', 'Sauber Ferrari',
       'Scuderia Toro Rosso', 'Marussia', 'Williams Cosworth',
       'Hrt Cosworth', 'Caterham Renault', 'Williams Renault',
       'Str Renault', 'Williams Mercedes', 'Mclaren Honda',
       'Haas Ferrari', 'Mclaren Renault', 'Alfa Romeo', 'Aston Martin'],
      dtype=object)

In [489]:
print(df["Detail_pitstop"].unique())
# Dropping Detail_pitstop since it contains only one value
df.drop(['Detail_pitstop'], axis=1, inplace=True)

['Pit-Stop-Summary']


Creating a function to change time to float

In [ ]:
# Function to convert time strings to float
def convert_time_to_float(time_str):
    if ':' in time_str:
        # Split into minutes and seconds
        minutes, seconds = map(float, time_str.split(':'))
        return minutes * 60 + seconds  # Convert to total seconds
    else:
        return float(time_str)  # Convert directly to float

df["Time_pitstop"] = df["Time_pitstop"].apply(convert_time_to_float)

In [ ]:
df["Total"] = df["Total"].apply(convert_time_to_float)

In [ ]:
df.to_csv('cleaned_merged_race_data.csv', index=False)